# 0. Install and Import dependencies

In [1]:
# Gym stuff
import gymnasium as gym
import gym_anytrading

# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# gym trading and finta
from gym_anytrading.envs import StocksEnv
from finta import TA

# 1. Data import

In [2]:
df = pd.read_csv('data/df_train.csv', index_col=0)
df.index = pd.to_datetime(df.index)

In [3]:
features = df.drop('Price', axis=1).columns.tolist()

## 2. Create the environment

In [4]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Price'].to_numpy()[start:end]
    signal_features = env.df.loc[:, features].to_numpy()[start:end]
    return prices, signal_features

In [5]:
class MyCustomEnv(StocksEnv):
    _process_data = add_signals
    
env2 = MyCustomEnv(df=df, window_size=96, frame_bound=(12,50))

# 3. Build Environment and Train

In [6]:
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [8]:
model = DQN('MlpPolicy', env, learning_rate=0.001, verbose=1)
model.learn(total_timesteps=1000, log_interval=10)

Using cpu device


C:\Users\BranimirRaguzHedgeGO\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 15.38GB > 2.61GB
  warnings.warn(


ValueError: could not broadcast input array from shape (0,20) into shape (96,20)

# 4. Evaluation

In [ ]:
env = MyCustomEnv(df=df, window_size=12, frame_bound=(63,100))
obs = env.reset()[0]

while True: 
    action, _states = model.predict(obs)
    obs, rewards, extra, done, info = env.step(action)
    if done:
        print("info", info)
        break

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()